## How to do math in Tensorflow (with logistic regression example)

[Tensorflow](https://www.tensorflow.org/) is a framework introduced by Google for machine learning, but most people perceive it a tool for building deep neural networks. But actually it can be used as a good mathematics symbolic framework for numeric calculation and making use of the computational power of GPUs and distributed clusters effortlessly.

Here I will present the classic example of logistic regression without using the built-in optimization tools.

### Generate simulated data 

In [2]:
import toy_data
import bokeh.io
bokeh.io.output_notebook()

x_dim = 2
gm = toy_data.GaussianMixture(n_class=2, dim=x_dim)
toy_data.visualize_2D(gm.Classes, gm.class_colors)

Loading BokehJS ...

In [3]:
import numpy as np
import tensorflow as tf

io_tf = lambda dim: tf.placeholder(tf.float32, dim)
var_tf = lambda dim: tf.Variable(dim)
def tf_logistic(_X):  
    return 1/(1 + tf.exp(-_X))

with tf.device('/gpu:0'):
    x = io_tf([None, x_dim])
    y = io_tf([None, 1])
    W = var_tf(tf.zeros([x_dim, 1]))
    b = var_tf(tf.zeros([1]))
    y_ = tf_logistic(tf.matmul(x, W) + b)
    loglikelihood = tf.reduce_sum(tf.log((1-y_)*(1-y) + y*y_))
    dydW = tf.gradients(loglikelihood, W)
    dydb = tf.gradients(loglikelihood, b)

lr = 0.1  # Learning rate
beta = 0.8 # Backtracking
alpha = 0.5 # Backtracking
data_X = gm.tr.X
data_y = np.array([gm.tr.y[:, 0]]).T
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    writer = tf.train.SummaryWriter('summary', sess.graph)
    sess.run(tf.initialize_all_variables())
    for i in range(100):
        l = sess.run(loglikelihood, {x:data_X, y:data_y})
        db = sess.run(dydb, {x:data_X, y:data_y})
        dw = sess.run(dydW, {x:data_X, y:data_y})

        W_old = W.eval()
        b_old = b.eval()
        while True:
            W.assign(W_old + lr*dw[0]).eval()
            b.assign(b_old + lr*db[0]).eval()
            ln = sess.run(loglikelihood, {x:data_X, y:data_y})
            norm_grad = np.sum(dw[0]**2) + db[0]**2
            if ln < l + lr*alpha*norm_grad:
                lr *= beta
            else:
                break
        
    def classifier(_X):
        _y = sess.run(y_, feed_dict={x: _X})
        _Y = np.column_stack((_y, 1 - _y))
        return _Y

    toy_data.visualize_2D(gm.Classes, gm.class_colors, classifyF=classifier, res=100)